In [118]:
import pandas as pd
import time
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

#run in incognito
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(chrome_options=chrome_options)

import warnings
warnings.filterwarnings('ignore')

In [70]:
driver.get('https://nomadlist.com/')

In [71]:
#scroll to botton of page to load all cities
# (https://www.codegrepper.com/search.php?q=selenium%20python%20scroll%20down%20until%20the%20end)
SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [72]:
#get city names
city_names = driver.find_elements_by_xpath('//h2[@class="itemName"]')

cty = []
for i in range(len(city_names)):
     cty.append(city_names[i].text)


In [ ]:
cty

In [74]:
#match name format of cities for later scrape using url per city
cty_list = []
for i in cty:
    a = i.split(",")
    cty_list.append(a[0].replace(" ", "-"))

In [76]:
len(cty_list)

1346

In [44]:
driver.get('https://nomadlist.com/lisbon')

In [13]:
# get column names
column_ = driver.find_elements_by_xpath('//td[@class="key"]')

column = []
for i in range(len(column_)):
     column.append(column_[i].text)

column

['⭐️ Total score',
 '👍 Quality of life score',
 '👶 Family score',
 '💵 Cost',
 '📡 Internet',
 '😝 Fun',
 '⛅️ Temperature (now)',
 '💦 Humidity (now)',
 '💨 Air quality (now)',
 '💨 Air quality (annual)',
 '👌 Safety',
 '👮\u200d♀️ Lack of crime*',
 '🤝 Lack of racism*',
 '🎓 Education level*',
 '💰 Income level*',
 '🙊 English speaking*',
 '😤 People density',
 '🚶 Walkability',
 '✌️ Peace (no pol. conflict)',
 '🚦 Traffic safety*',
 '🏥 Hospitals*',
 '😄 Happiness*',
 '🍸 Nightlife',
 '📶 Free WiFi in city',
 '🖥 Places to work from',
 '❄️ A/C or heating',
 '😁 Friendly to foreigners',
 '🗯 Freedom of speech*',
 '👩 Female friendly',
 '🌈 LGBTQ+ friendly',
 '🎅 Startup Score',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:
"""
function to loop trough cities urls and get the cotent 
"""

# city_info = []
# for c in cty_list:
#     driver.get(f"https://nomadlist.com/{c}")
#     time.sleep(2)
#     content_ = driver.find_elements_by_xpath('//td[@class="value"]')
#     time.sleep(2)
#     cnt = []
#     for i in range(len(content_)):
#         cnt.append(content_[i].text)
#     print(len(cnt))
#     city_info.append(cnt)


In [77]:
columns = ['⭐️ Total score',
 '👍 Quality of life score',
 '👶 Family score',
 '💵 Cost',
 '📡 Internet',
 '😝 Fun',
 '⛅️ Temperature (now)',
 '💦 Humidity (now)',
 '💨 Air quality (now)',
 '💨 Air quality (annual)',
 '👌 Safety',
 '👮\u200d♀️ Lack of crime*',
 '🤝 Lack of racism*',
 '🎓 Education level*',
 '💰 Income level*',
 '🙊 English speaking*',
 '😤 People density',
 '🚶 Walkability',
 '✌️ Peace (no pol. conflict)',
 '🚦 Traffic safety*',
 '🏥 Hospitals*',
 '😄 Happiness*',
 '🍸 Nightlife',
 '📶 Free WiFi in city',
 '🖥 Places to work from',
 '❄️ A/C or heating',
 '😁 Friendly to foreigners',
 '🗯 Freedom of speech*',
 '👩 Female friendly',
 '🌈 LGBTQ+ friendly',
 '🎅 Startup Score']

In [78]:
#get cities content 
raw_data = []
for c in cty_list:
    driver.get(f"https://nomadlist.com/{c}")
    final_row = {}
    time.sleep(2)
    for col in columns:
        try:
            single_cat = (driver.find_element_by_xpath(f"//tr[contains(., '{col}')]")).text
            final_row.update({col:single_cat})    
        except:
            final_row.update({col:"None"})
    raw_data.append(final_row)

In [ ]:
raw_data

In [100]:
df = pd.DataFrame(data=raw_data, columns=columns)

In [ ]:
pd.set_option('display.max_rows', None)
df

In [102]:
df.insert(0, 'City', cty)

In [104]:
`#save to csv
df.to_csv('nomad_list.csv')

In [121]:
df = pd.read_csv('nomad_list.csv')

In [147]:
df_none = df.loc[df['⭐️ Total score'] == 'None']

In [148]:
# cities for which scrape did not work 
df_none['City']

10                       São Paulo
23                        Medellín
25                             NaN
51                          Kraków
108              Portimão, Algarve
268                        Wrocław
288                          Malmö
289                       San José
313                 St. Catharines
357                        Vitória
387                     Nuku'alofa
429                     Valparaíso
450     San Cristóbal de las Casas
485                 St. Joseph, MO
533                  St. Louis, MO
588            São José dos Campos
593                        Jundiaí
750                   Viña del Mar
768              Coeur d'Alene, ID
770             Port St. Lucie, FL
818                  St. Cloud, MN
1006                        Sana'a
1021                       Huai'an
1066                St. George, UT
1087                          León
1098                        Rui'an
1104                       Šumperk
1198                        Dorobō
1232                